In [1]:
from __future__ import division, absolute_import, print_function
%matplotlib inline

## Get the data

This is the simulated data used in the DeepLIFT paper. The the pwms for the GATA motif and TAL motif (GATA_disc1 and TAL1_known1 from http://compbio.mit.edu/encode-motifs/) are used to generate motifs which are inserted into a random background. Sequences containing at least one GATA motif are a 1 for task 1, 0 otherwise. Sequences containing at least one TAL motif are a 1 for task 2, 0 otherwise. Sequences containing both a TAL and a GATA motif are a 1 for task 0. 

In [2]:
!./grab_data.sh

File sequences.simdata.gz exists already


In [3]:
try:
    import simdna
except ImportError, e:
    print("installing simdna package")
    !pip install -e "git://github.com/kundajelab/simdna.git@0.4.0#egg=simdna"
    print("\n******************************************************************************")
    print("RESTART THE JUPYTER KERNEL TO PICK UP ON THE INSTALLATION!!!")
    print("******************************************************************************")

In [4]:
import simdna.synthetic as synthetic
import gzip

data_filename = "sequences.simdata.gz"
data = synthetic.read_simdata_file(data_filename)

One-hot encode the sequence data into a 3d array, where the last axis ("channel" axis) is the ACGT axis.

In [5]:
import numpy as np

def one_hot_encode_along_channel_axis(sequence):
    #theano dim ordering, uses row axis for one-hot
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1
            
onehot_data = np.array([one_hot_encode_along_channel_axis(seq) for seq in data.sequences])
print(onehot_data.shape)

(8000, 200, 4)


## Compute the gapped kmer embeddings

We will compute the embeddings using the GPU to scan for matches to the gapped kmers, allowing for some numbed of mismatches. First, we will prepare the function that is going to do our scanning on the GPU.

In [6]:
import ssvmimp
import ssvmimp.train

max_mismatches=0

filters, string_reps, embedding_func = ssvmimp.train.get_gapped_kmer_embedding_filters_and_func(
                                kmer_len=6, alphabet=['A','C','G','T'],
                                num_gaps=1, max_mismatches=max_mismatches)

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5005)


'string_reps' stores string representations of the filters, as shown below (gaps are indicated with whitespace):

In [7]:
print("Number of filters:",len(string_reps))
print("First ten filters:")
print("\n".join(string_reps[:10]))
print("Last ten filters:")
print("\n".join(string_reps[-10:]))

Number of filters: 5120
First ten filters:
AAAAA 
AAAAC 
AAAAG 
AAAAT 
AAACA 
AAACC 
AAACG 
AAACT 
AAAGA 
AAAGC 
Last ten filters:
T TTCG
T TTCT
T TTGA
T TTGC
T TTGG
T TTGT
T TTTA
T TTTC
T TTTG
T TTTT


Now, we compute the embeddings, which are the sum of the number of matches to each filter per sequence, accounting for the desired number of mismatches

In [8]:
embeddings = embedding_func(onehot=onehot_data, batch_size=20, progress_update=500)
print("Shape of the embeddings matrix:",embeddings.shape)

Done 0
Done 500
Done 1000
Done 1500
Done 2000
Done 2500
Done 3000
Done 3500
Done 4000
Done 4500
Done 5000
Done 5500
Done 6000
Done 6500
Done 7000
Done 7500
Shape of the embeddings matrix: (8000, 5120)


Limit features to gkmers that are statistically overrepresented in the positive set

In [9]:
train_set_num = 6000 #6000 examples will be used in the training set
train_embeddings = embeddings[:train_set_num]
kmer_counts_positives = np.sum(train_embeddings[data.labels[:train_set_num,0]==1]>0,axis=0)
kmer_counts_negatives = np.sum(train_embeddings[data.labels[:train_set_num,0]==0]>0,axis=0)
total_positives = np.sum(data.labels[:train_set_num,0])
total_negatives = np.sum(1-data.labels[:train_set_num,0])

In [10]:
import ssvmimp.stats.stats
reload(ssvmimp.stats.stats)

significant_kmer_indices = []
for i,(kmer_count_pos, kmer_count_neg) in enumerate(zip(kmer_counts_positives, kmer_counts_negatives)):
    total = total_positives + total_negatives
    special = total_positives
    picked = kmer_count_pos+kmer_count_neg
    specialPicked = kmer_count_pos
    result = ssvmimp.stats.stats.proportionTest(
                total=total, special=special,
                picked=picked, specialPicked=specialPicked)
    if (result.pval < 0.05):
        significant_kmer_indices.append(i)

print(len(significant_kmer_indices))

530


Normalize the embeddings so that the rows have unit magnitude. Only consider overrepresented kmers

In [11]:
normalized_embeddings = (embeddings[:,significant_kmer_indices]/
                         np.linalg.norm(embeddings[:,significant_kmer_indices], axis=1)[:,None])
print("Normalized embeddings shape:",normalized_embeddings.shape)

Normalized embeddings shape: (8000, 530)


## Train SVMs given the embeddings

In [12]:
import sklearn.svm
import sys

print("Training linear SVM for t0")
sys.stdout.flush()
t0_linear_classifier = sklearn.svm.LinearSVC().fit(X=normalized_embeddings[:train_set_num], y=data.labels[:train_set_num,0])
preds = t0_linear_classifier.predict(normalized_embeddings[train_set_num:])
print("Linear SVM accuracy for task",0,":",np.sum(data.labels[train_set_num:,0] == preds)/len(preds))

Training linear SVM for t0
Linear SVM accuracy for task 0 : 0.857


In [13]:
import sklearn.svm
t0_gaussian_classifier = sklearn.svm.SVC(C=1.0, kernel="rbf", gamma=1.0).fit(
                X=normalized_embeddings[:train_set_num],
                y=data.labels[:train_set_num,0])
preds = t0_gaussian_classifier.predict(normalized_embeddings[train_set_num:])
print("Gaussian SVM test accuracy for task",0,":",np.sum(data.labels[train_set_num:,0] == preds)/len(preds))
train_preds = t0_gaussian_classifier.predict(normalized_embeddings[:train_set_num])
print("Gaussian SVM train accuracy for task",0,":",np.sum(data.labels[:train_set_num,0] == train_preds)/len(train_preds))
print("Number of support vectors:",t0_gaussian_classifier.n_support_)

Gaussian SVM test accuracy for task 0 : 0.9485
Gaussian SVM train accuracy for task 0 : 0.9721666666666666
Number of support vectors: [1213  959]
